In [ ]:
import json
from together import Together
from openai import OpenAI

# togetherai_key = <insert togetherai key>
together_client = Together(api_key=togetherai_key)

# openai_key = <insert openai key>
client = OpenAI(api_key=openai_key)

In [ ]:
# Loading all JSONs

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/ref_extractions.json', 'r') as f:
    ref_extractions = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/case_summary_pairs_list.json', 'r') as f:
    case_summary_pairs_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_extrac_list.json', 'r') as f:
    citation_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_pmid_list.json', 'r') as f:
    matching_uids_pmid_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/o3_mini_response_list.json', 'r') as f:
    o3_mini_response_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/indices_list.json', 'r') as f:
    citation_valid_indices = json.load(f)


In [5]:
# ref_extractions = [
#     extract_single_ref(xml_pair, pmid_pair) 
#     for (xml_pair, pmid_pair) in zip(matching_uids_xml_paths, matching_uids_pmid_list)
# ]



In [4]:
chosen_extraction = citation_list[1]

In [76]:
chosen_extraction

{'citing_title': 'Agenesis of permanent canines: Rare case report',
 'citing_abstract': 'Agenesis of permanent canines is a rare condition, and that of both maxillary and mandibular permanent canines is extremely rare. It may occur either isolated or in association with other dental anomalies. Reports of such cases are very scarce in the literature. Need for early diagnosis of such conditions should be emphasized because of functional, esthetic, and psychological problems which should be evaluated and treated appropriately. The present paper presents a report of bilaterally missing permanent maxillary and mandibular canines. This case might contribute in the future studies of incidence of agenesis of permanent canines.',
 'citation_paragraphs': [],
 'cited_title': 'Permanent Maxillary Canine Agenesis: A Rare Case Report',
 'cited_abstract': 'ABSTRACT Congenitally missing teeth (CMT) are among one of the commonly known dental anomalies. The most frequently missing teeth in the permanent

In [77]:
prompt_exc_few_shot = """You are a medical citation analysis assistant. You will read excerpts from a 'citing paper' referencing another 'cited paper,' each describing a single-patient case report. Your task is to produce four pieces of information:

1. Reason for Citation (a short phrase or sentence about why the authors mention the cited paper).
2. Possible Similarities (a brief mention of how these patients’ cases align, if at all).
3. Explanation (one to two sentences summarizing how the citation is used).
4. Confidence (High, Medium, or Low), based on the degree of similarity between the single-patient cases:
- High: The text explicitly states that the two patients share significant clinical or therapeutic similarities.
- Medium: Some parallels are hinted at but lack explicit detail or direct confirmation.
- Low: The citation is vague or does not clearly connect the patients’ key features.

----
Follow this exact output format:
Reason for Citation: <short phrase or sentence>
Possible Similarities: <brief mention>
Explanation: <one to two sentences>
Confidence: <High/Medium/Low>

----
Read the following excerpt(s) and generate your final answer in the required format. Unify your response if multiple citations are present. If no citations are specified, base your response on the articles' titles and abstracts.
"""

In [75]:

completion = client.chat.completions.create(
    model="o3-mini",
    reasoning_effort = "medium",
    messages=[
        {
            "role": "developer",
            "content": f"{prompt_exc_few_shot}"
        },
        {
            "role": "user",
            "content": f"{chosen_extraction}"
        }
    ],
)

response = completion.choices[0].message.content.strip()

print(response)

Reason for Citation: To reference a documented case of rare maxillary canine agenesis.  
Possible Similarities: Both case reports describe bilateral absence of permanent maxillary canines.  
Explanation: The citing paper uses the cited report as an example of an extremely rare dental anomaly involving bilateral maxillary canine agenesis, thereby placing its case report in the context of existing literature on rare dental agenesis. Although the additional dental anomalies differ between the cases, the focus on missing maxillary canines creates a relevant parallel.  
Confidence: Medium


In [23]:
o3_mini_response_list[0]

'Reason for Citation: To illustrate a similar case of choroidal osteoma complicated by secondary choroidal neovascularization treated with ranibizumab.  \nPossible Similarities: Both cases involve female patients with choroidal osteoma developing CNV who were successfully managed with intravitreal ranibizumab.  \nExplanation: The citation is used to show a comparable instance where a patient with choroidal osteoma developed secondary choroidal neovascularization, and treatment with ranibizumab led to regression of the CNV and improvement in visual acuity. The similarities in clinical presentation and treatment outcomes underscore the relevance of the cited work to the case presented.  \nConfidence: High.'

In [5]:
def calc_indices_summaries_list(case_summary_pairs_list):
    """
    Returns two lists:
      - A list of indices corresponding to elements depicting both cases.
      - A list of indices corresponding to elements depicting less than 2 cases.
    """
    valid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) == 2]
    invalid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) != 2]

    return valid_indices, invalid_indices



In [6]:
len(o3_mini_response_list)

31080

In [7]:
len(citation_valid_indices)

31080

In [8]:
summaries_valid_indices, summaries_invalid_indices = calc_indices_summaries_list(case_summary_pairs_list)



In [8]:
o3_mini_response_list[0]

'Reason for Citation: To illustrate a similar case of choroidal osteoma complicated by secondary choroidal neovascularization treated with ranibizumab.  \nPossible Similarities: Both cases involve female patients with choroidal osteoma developing CNV who were successfully managed with intravitreal ranibizumab.  \nExplanation: The citation is used to show a comparable instance where a patient with choroidal osteoma developed secondary choroidal neovascularization, and treatment with ranibizumab led to regression of the CNV and improvement in visual acuity. The similarities in clinical presentation and treatment outcomes underscore the relevance of the cited work to the case presented.  \nConfidence: High.'

In [9]:
def find_common_indices(summaries_valid_indices, citation_valid_indices):
    """
    Find the common indices between summaries_valid_indices and citation_valid_indices:
      - summaries_valid_indices - represents the indices corresponding to successfully extracted case summaries (0-31,691) 
      - citation_valid_indices - represents the indices corresponding to successfully cited-citing pairs (0-31,691)
    """
    summaries_valid_indices_set = set(summaries_valid_indices)
    citation_valid_indices_set = set(citation_valid_indices)
    common_indices = []

    for i in range(len(matching_uids_pmid_list)):
        if i in summaries_valid_indices_set and i in citation_valid_indices_set:
            common_indices.append(i)
    
    return common_indices


common_indices = find_common_indices(summaries_valid_indices, citation_valid_indices)

In [30]:
print(len(o3_mini_response_list))
print(len(citation_list))

31080
31080


In [10]:
common_indices[1000]

1030

In [14]:
citation_valid_indices[1004]

1030

In [18]:
citation_valid_indices_dict[1030]

1004

In [ ]:
citation_valid_indices_dict = {citation_valid_indices[idx]: idx for idx in range(len(citation_valid_indices))}

In [ ]:
curr_index = common_indices[4000]

case1 = case_summary_pairs_list[curr_index][0]

case2 = case_summary_pairs_list[curr_index][1]

citation_reason = o3_mini_response_list[citation_valid_indices_dict[curr_index]]

question_generating_prompt = """
You are an experienced clinician specializing in medical imaging. You will be given two cases and the reason for citation between the two (one case report cites the other), and asked to generate a set of comparative, clinically relevant questions that highlight both similarities and differences, helping differentiate potential diagnoses. Your questions should:

1. **Focus on Diagnostic Clues and Clinical Significance:**
   - Avoid trivial or purely technical details (like minor differences in image orientation).
   - Emphasize findings that inform diagnosis (e.g., presence of lesions, pattern of abnormalities, symptoms).

2. **Use a Comparative Framing:**
   - Explicitly compare Case A and Case B.
   - Aim for questions that probe both common ground and distinguishing features.
   - If possible, try asking mostly yes/no questions.

3. **Follow a Hierarchical Reasoning Approach:**
   - **Level 1 (Broad Context)**: Are both cases the same modality/organ system?
   - **Level 2 (General Diagnosis Category)**: Are they both infectious, both neoplastic, etc.?
   - **Level 3 (Specific Features & Findings)**: Detailed imaging findings (e.g., cavitation, consolidation, nodules), clinical presentation, organism type, etc.

4. **Consider Step-by-Step Reasoning:**
   - You may first silently analyze the diagnoses and imaging findings for each case.
   - Then generate questions that a clinician would naturally ask to tease out whether the two cases have the same underlying pathology or different pathologies.

5. **Provide a subtitle specifying the level (1/2/3) of the current set of questions.**
"""

question_generating_user = f"""
Below are two cases and the reason for citation between the two. Read them carefully, then generate a concise list of question prompts (in bullet or numbered format) that compare these cases on clinically meaningful aspects.

---
Case A:
{case1}

Case B:
{case2}

Reason for Citation, Possible Similarities and Explanation:
{citation_reason}
---

Now, **generate the list of comparative, clinically relevant questions** that a radiologist or physician might ask to determine whether these two cases share a diagnosis or differ in key features. 

Remember:
- Maintain clinical depth: mention imaging findings, pathophysiology, or hallmark symptoms.
- Emphasize similarities/differences that impact diagnosis.
- Keep the questions focused and direct, as if you are performing a differential diagnosis.
- Avoid vague or generic questions; ensure each question contributes to understanding diagnostic overlap or divergence.
- Start with broad-level questions (anatomy, broad diagnosis similarity), then proceed to more specific questions about the nature of the condition, causative agents, imaging signs, etc.
- Adhere to yes/no questions, if possible.
- Provide a subtitle specifying the level (1/2/3) of each set of questions.

Provide your final list of questions now.
"""

In [37]:

completion = client.chat.completions.create(
    model="o3-mini",
    reasoning_effort = "low",
    messages=[
        {
            "role": "developer",
            "content": f"{question_generating_prompt}"
        },
        {
            "role": "user",
            "content": f"{question_generating_user}"
        }
    ],
)

generated_questions = completion.choices[0].message.content.strip()

print(generated_questions)

Level 1 – Broad Context (Anatomy and Modality):
1. Do both Case A and Case B involve lesions in the mandibular region?
2. Are both cases assessed using radiographic imaging modalities (panoramic radiography in Case A and cone beam CT in Case B)?

Level 2 – General Diagnostic Category:
3. Do both cases present with cementoblastomas as the final diagnostic outcome?
4. Are both patients pediatric males presenting with lesions involving primary (deciduous) molar regions?
5. Considering the overall clinical findings, do both cases suggest a benign osseous neoplasm rather than an aggressive or malignant process?

Level 3 – Specific Imaging Findings and Features:
6. Do both cases demonstrate a well-defined radiopaque mass with a peripheral radiolucent rim on imaging studies?
7. In both cases, is there evidence of cortical expansion (buccal and/or lingual) associated with the lesion?
8. Are reversal lines and cementum-like tissue features identified histopathologically in both cases?
9. Do bot

In [93]:
citation_valid_indices_dict[1000]

1026

In [94]:
common_indices[1000]

1030

In [15]:
citation_valid_indices_dict = {citation_valid_indices[idx]: idx for idx in range(len(citation_valid_indices))}

In [96]:
citation_valid_indices_dict[1030]

1004

In [ ]:
def compute_tokens_average(citation_valid_indices, case_summary_pairs_list, o3_mini_response_list, common_indices):
    
    counter = 0
    citation_valid_indices_dict = {citation_valid_indices[idx]: idx for idx in range(len(citation_valid_indices))}

    for i in common_indices:

        case1 = case_summary_pairs_list[i][0]

        case2 = case_summary_pairs_list[i][1]

        citation_reason = o3_mini_response_list[citation_valid_indices_dict[i]]

        question_generating_prompt = """
        You are an experienced clinician specializing in medical imaging. You will be given two cases and the reason for citation between the two (one case report cites the other), and asked to generate a set of comparative, clinically relevant questions that highlight both similarities and differences, helping differentiate potential diagnoses. Your questions should:

        1. **Focus on Diagnostic Clues and Clinical Significance:**
        - Avoid trivial or purely technical details (like minor differences in image orientation).
        - Emphasize findings that inform diagnosis (e.g., presence of lesions, pattern of abnormalities, symptoms).

        2. **Use a Comparative Framing:**
        - Explicitly compare Case A and Case B.
        - Aim for questions that probe both common ground and distinguishing features.
        - If possible, try asking mostly yes/no questions.

        3. **Follow a Hierarchical Reasoning Approach:**
        - **Level 1 (Broad Context)**: Are both cases the same modality/organ system?
        - **Level 2 (General Diagnosis Category)**: Are they both infectious, both neoplastic, etc.?
        - **Level 3 (Specific Features & Findings)**: Detailed imaging findings (e.g., cavitation, consolidation, nodules), clinical presentation, organism type, etc.

        4. **Consider Step-by-Step Reasoning:**
        - You may first silently analyze the diagnoses and imaging findings for each case.
        - Then generate questions that a clinician would naturally ask to tease out whether the two cases have the same underlying pathology or different pathologies.

        5. **Provide a subtitle specifying the level (1/2/3) of the current set of questions.**
        """

        question_generating_user = f"""
        Below are two cases and the reason for citation between the two. Read them carefully, then generate a concise list of question prompts (in bullet or numbered format) that compare these cases on clinically meaningful aspects.

        ---
        Case A:
        {case1}

        Case B:
        {case2}

        Reason for Citation, Possible Similarities and Explanation:
        {citation_reason}
        ---

        Now, **generate the list of comparative, clinically relevant questions** that a radiologist or physician might ask to determine whether these two cases share a diagnosis or differ in key features. 

        Remember:
        - Maintain clinical depth: mention imaging findings, pathophysiology, or hallmark symptoms.
        - Emphasize similarities/differences that impact diagnosis.
        - Keep the questions focused and direct, as if you are performing a differential diagnosis.
        - Avoid vague or generic questions; ensure each question contributes to understanding diagnostic overlap or divergence.
        - Start with broad-level questions (anatomy, broad diagnosis similarity), then proceed to more specific questions about the nature of the condition, causative agents, imaging signs, etc.
        - Adhere to yes/no questions, if possible.

        Provide your final list of questions now.
        """

        prompt = question_generating_prompt + "\n" + question_generating_user
        model_name = "gpt-4o"
        prompt_count, response_count = count_tokens(prompt, "", model_name)
        counter += prompt_count

    return counter / len(common_indices)
    
tokens_avg = compute_tokens_average(citation_valid_indices, case_summary_pairs_list, o3_mini_response_list, common_indices)


In [41]:
prompt = answering_prompt + "\n" + answering_user
model_name = "gpt-4o"
prompt_count, response_count = count_tokens(prompt, "", model_name)


In [42]:
prompt_count

2582

In [40]:
answering_user

"\nBelow are the two case descriptions (with the same details as before) and the set of questions to be answered.\n\n---\nCase A:\nA 4.5-year-old boy was referred to the Oral and Maxillofacial Surgery Department of Tehran University of Medical Sciences with the chief complaint of swelling and mild pain in the posterior right mandibular area. An informed signed consent was obtained from the parent of the patient. The familial and medical histories were not contributory, and no evidence of facial trauma was reported.\n On extra oral examination, there was mild swelling of the right lower jaw leading to mild facial asymmetry. The swelling was bony hard in consistency.\n An intraoral examination revealed buccal and lingual cortical expansion. The overlying mucosa was normal. A vitality test with an electrical pulp tester yielded a positive reaction for the deciduous mandibular second molar. There was no functional difficulty with speech, swallowing, and breathing.\n On panoramic radiograph

In [22]:
tokens_avg

2323.516773111412

In [91]:
citation_valid_indices_dict[31079]

31691

In [84]:
citation_valid_indices[0:5]

[0, 1, 2, 3, 4]

In [75]:
len(case_summary_pairs_list)

31692

In [76]:
len(o3_mini_response_list)

31080

In [77]:
len(common_indices)

31002

In [80]:
citation_valid_indices[1000]

1026

In [78]:
common_indices[1000]

1030

In [73]:
prompt = question_generating_prompt + " " + question_generating_user
# response = response
model_name = "gpt-4o"
prompt_count, response_count = count_tokens(prompt, "", model_name)
print(f"Prompt tokens: {prompt_count}")
print(f"Response tokens: {response_count}")


Prompt tokens: 1785
Response tokens: 0


In [24]:

completion = client.chat.completions.create(
    model="o3-mini",
    reasoning_effort = "low",
    messages=[
        {
            "role": "developer",
            "content": f"{question_generating_prompt}"
        },
        {
            "role": "user",
            "content": f"{question_generating_user}"
        }
    ],
)

generated_questions = completion.choices[0].message.content.strip()

print(generated_questions)

Level 1 – Broad Anatomy and Modality  
1. Do both Case A and Case B involve a choroidal osteoma diagnosed via clinical examination and multimodal imaging (e.g., fundus photography, B-scan ultrasonography, CT/OCT/FFA)?  
2. Were both cases evaluated with imaging studies that demonstrated a hyperechoic choroidal lesion causing acoustic shadowing?

Level 2 – General Diagnosis Category  
3. Are both cases diagnosed with choroidal osteoma complicated by secondary choroidal neovascularization?  
4. Did both patients present with decreased visual acuity in the affected eye that warranted treatment with intravitreal ranibizumab?

Level 3 – Specific Imaging Findings and Clinical Features  
5. Does Case A display additional features such as decalcification and choroidal atrophy on follow-up that are not specifically mentioned in Case B?  
6. Is the lesion in Case A described as geographic-shaped and peripapillary with involvement of the macula, while Case B’s lesion is described as well-defined 

In [29]:
generated_questions = """
Subtitle: Level 1 – Broad Context and Diagnosis
1. Do both cases involve the same anatomical region (i.e., the choroid and macula) on ocular imaging?  
2. Are both cases diagnosed with choroidal osteoma complicated by secondary choroidal neovascularization?

Subtitle: Level 2 – General Diagnostic Category and Treatment Approach
3. Do both cases exhibit typical imaging features of choroidal osteoma on B-scan ultrasound (e.g., hyperechoic lesion with acoustic shadowing)?  
4. Was intravitreal ranibizumab administered as the primary treatment for CNV in both cases?  
5. Did both patients show regression of subretinal fluid on OCT following anti-VEGF therapy?

Subtitle: Level 3 – Specific Imaging Findings and Clinical Nuances
6. Does Case A show evidence of lesion growth and partial decalcification that is not described in Case B?  
7. Are the patterns of fluorescein angiography similar in both cases, with early hyperfluorescence and late staining, despite minor differences in CNV visualization?  
8. Is the location of the lesion comparable between the cases, with Case A involving the peripapillary area and macula versus Case B predominantly affecting the macula?  
9. Are the OCT/OCTA findings of CNV in Case A more detailed (e.g., showing choriocapillaris atrophy and layered neovascularization) compared to the OCT findings in Case B?  
10. Did both cases experience improvement in visual acuity after ranibizumab treatment, despite differences in the degree of initial visual loss and specific follow‐up intervals?
"""

In [39]:
answering_prompt = """
You are an experienced clinician specializing in medical imaging. Previously, a list of comparative, clinically oriented questions was generated for two separate cases (Case A and Case B). Now, your task is to answer those questions in a way that highlights the diagnostic similarities and differences between the two cases.

Instructions for answering:

1. **Refer to Both Cases**  
   - You will be given the same case details for Case A, Case B, Reason for Citation, along with the full list of questions.
   - Answer each question by explicitly referencing the relevant details from Case A and Case B that support your conclusion.

2. **Structure of Answers**  
   - For yes/no questions, begin your response with a clear “Yes” or “No,” then provide a brief rationale.
   - For open-ended or more detailed comparative questions, give a concise yet clinically thorough answer.
   - If you need to highlight differences in imaging findings, symptoms, or etiologies, do so clearly and directly.
   - Split each answer to 'In case A, ...' and 'In case B, ...'

3. **Clinical Depth & Relevance**  
   - Focus on the clinical significance of each answer, referencing imaging features, pathophysiology, or typical clinical presentation.
   - Avoid tangential or purely theoretical discussion; make sure each answer directly relates to the question and helps differentiate or confirm diagnoses.

4. **Accuracy & Clarity**  
   - Base your answers on standard radiological and clinical knowledge, ensuring that any assertions are consistent with typical findings and established practice.
   - If you must make an assumption (e.g., missing data), state it explicitly and provide your best inference based on standard clinical reasoning.

5. **Format**  
   - Number your answers to match the questions.
   - Keep them concise, but complete enough to stand as a definitive answer.
"""

answering_user = f"""
Below are the two case descriptions (with the same details as before) and the set of questions to be answered.

---
Case A:
{case1}

Case B:
{case2}

Reason for Citation, Possible Similarities and Explanation:
{citation_reason}

Questions:
{generated_questions}
---

Provide your answers now:
"""


In [31]:

completion = client.chat.completions.create(
    model="o3-mini",
    reasoning_effort = "low",
    messages=[
        {
            "role": "developer",
            "content": f"{answering_prompt}"
        },
        {
            "role": "user",
            "content": f"{answering_user}"
        }
    ],
)

answers = completion.choices[0].message.content.strip()

print(answers)

1. Yes.  
 • In case A, the lesion involves the choroid with peripapillary extension and macular involvement, as noted by the fundus photography and OCT findings.  
 • In case B, the choroidal osteoma is specifically seen at the macula on fundus examination and OCT.

2. Yes.  
 • In case A, the patient was diagnosed with choroidal osteoma that later developed secondary choroidal neovascularization, confirmed by OCTA and other imaging modalities.  
 • In case B, the clinical and imaging findings (FFA, OCT, and B-scan USG) established the diagnosis of choroidal osteoma complicated by secondary CNV.

3. Yes.  
 • In case A, B-scan ultrasonography revealed a dense echogenic (hyperechoic) plaque with acoustic shadowing, which is typical for choroidal osteoma.  
 • In case B, B-scan USG similarly demonstrated a hyperechoic choroidal lesion causing acoustic shadowing.

4. Yes.  
 • In case A, intravitreal ranibizumab was administered following the detection of CNV, leading to improvement in O

In [20]:
## Counts the number of tokens in prompts/responses, for a specified model's tokenizer ### 

import tiktoken

def count_tokens(prompt_text: str, response_text: str, model_name: str) -> tuple:
    """
    Counts the number of tokens in the prompt_text (input) and response_text (output)
    using the specified model's tokenizer.
    
    Args:
        prompt_text (str): The input/prompt text.
        response_text (str): The output/response text.
        model_name (str): Name of the model tokenizer to use, e.g. 'gpt-3.5-turbo', 'gpt-4', etc.
    
    Returns:
        tuple: (prompt_token_count, response_token_count)
    """
    # Get the tokenizer for the specified model
    tokenizer = tiktoken.encoding_for_model(model_name)
    
    # Encode the input prompt and output response into tokens
    prompt_tokens = tokenizer.encode(prompt_text)
    response_tokens = tokenizer.encode(response_text)
    
    # Return the token counts
    return len(prompt_tokens), len(response_tokens)

# prompt = messages[0]['content'] + " " + messages[1]['content']
prompt = question_generating_prompt + " " + question_generating_user
# response = response
model_name = "gpt-4o"
prompt_count, response_count = count_tokens(prompt, "", model_name)
print(f"Prompt tokens: {prompt_count}")
print(f"Response tokens: {response_count}")


NameError: name 'question_generating_prompt' is not defined

In [9]:
### TogetherAI Llama and DeepSeek ###

# # Llama 3.3-70B, including few-shot
# llama_response = together_client.chat.completions.create(
#     model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
#     messages=[
#         {
#             "role": "system",
#             "content": prompt_inc_few_shot_1
#         },
#         {
#             "role": "user",
#             "content": f"Here is the information you have:\n\n {chosen_extraction}"
#         }
#     ],
#     max_tokens=1000,
#     temperature=0.5,
#     top_p=0.8,
# )

# # Extract and print the generated content
# print(llama_response.choices[0].message.content)


# # DeepSeek V3, including few-shot
# deepseek_response = together_client.chat.completions.create(
#     model="deepseek-ai/DeepSeek-V3",
#     messages=[
#         {
#             "role": "system",
#             "content": f"{prompt_inc_few_shot_1}"
#         },
#         {
#             "role": "user",
#             "content": f"Here is the information you have:\n\n {chosen_extraction}"
#         }
#     ],
#     max_tokens=1000,
#     temperature=0.5,
#     top_p=0.8,
# )

# # Extract and print the generated content
# print(deepseek_response.choices[0].message.content)

In [11]:
def view_nxml_content(file_path):
    """
    Opens and prints the raw contents of the .nxml file.
    
    :param file_path: Path to the .nxml file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)


In [6]:
import os

def get_xml_path(pmc_id: str):
    pmc_id = pmc_id[:-2]

    path = f'/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC{pmc_id}/{pmc_id}_1'

    for file_name in os.listdir(path):
        if file_name.endswith('.nxml'):
            return os.path.join(path, file_name)
    return None

get_xml_path('8167975-1')


'/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC8167975/8167975_1/12886_2021_Article_2004.nxml'

In [12]:
# matching_uids_xml_paths = [
#     [get_xml_path(uid) for uid in pair]
#     for pair in matching_uids
# ]

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)


In [ ]:
#!/usr/bin/env python3

import os
import sys
import json
import re
import string
from lxml import etree

##############################################################################
# 1. parse_nxml: gather pmid, doi, pmcid, main title, abstract, references
##############################################################################

def parse_nxml(file_path):
    """
    Parses a JATS .nxml file to gather:
      - pmid, doi, pmcid
      - the main article title, abstract
      - references => { ref_id -> { "doi":..., "pmid":..., "pmcid":..., "title":... } }
      - the parsed lxml 'tree'

    If file not found or parse error => returns None.
    """
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        return None

    parser = etree.XMLParser(recover=True)
    try:
        tree = etree.parse(file_path, parser)
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None

    root = tree.getroot()

    def _get_article_id(id_type):
        elem = root.find(f'.//article-id[@pub-id-type="{id_type}"]')
        return elem.text.strip() if elem is not None and elem.text else None

    paper_pmid  = _get_article_id("pmid")
    paper_doi   = _get_article_id("doi")
    paper_pmcid = _get_article_id("pmc")

    # Extract main article title
    title_elems = root.xpath('//*[local-name()="title-group"]//*[local-name()="article-title"]')
    if not title_elems:
        title_elems = root.xpath('//*[local-name()="article-title"]')
    paper_title = "".join(title_elems[0].itertext()).strip() if title_elems else ""

    # Extract abstract
    abstract_elems = root.xpath('//*[local-name()="abstract"]')
    abs_texts = []
    for abs_elem in abstract_elems:
        txt = " ".join(abs_elem.itertext()).strip()
        if txt:
            abs_texts.append(txt)
    paper_abstract = " ".join(abs_texts)

    # Extract references => store pmid, doi, pmcid, plus a "title" from <article-title> or fallback
    references = {}
    ref_list = root.xpath('//*[local-name()="ref"]')
    for ref_node in ref_list:
        r_id = ref_node.get('id')
        if not r_id:
            continue

        # gather pub-ids
        doi_elems   = ref_node.xpath('.//*[local-name()="pub-id"][@pub-id-type="doi"]')
        pmid_elems  = ref_node.xpath('.//*[local-name()="pub-id"][@pub-id-type="pmid"]')
        pmcid_elems = ref_node.xpath('.//*[local-name()="pub-id"][@pub-id-type="pmc"]')

        cited_doi   = doi_elems[0].text.strip()   if doi_elems   and doi_elems[0].text else None
        cited_pmid  = pmid_elems[0].text.strip()  if pmid_elems  and pmid_elems[0].text else None
        cited_pmcid = pmcid_elems[0].text.strip() if pmcid_elems and pmcid_elems[0].text else None

        # Attempt <article-title> in <element-citation>, else fallback <mixed-citation>, else <comment>
        ec_title_elems = ref_node.xpath('.//*[local-name()="element-citation"]/*[local-name()="article-title"]')
        if ec_title_elems:
            ref_title = " ".join(ec_title_elems[0].itertext()).strip()
        else:
            mixed_elems = ref_node.xpath('.//*[local-name()="mixed-citation"]')
            if mixed_elems:
                ref_title = extract_title_from_mixed_citation(mixed_elems[0])
            else:
                # final fallback => <comment>
                comment_elems = ref_node.xpath('.//*[local-name()="element-citation"]/*[local-name()="comment"]')
                if comment_elems:
                    ref_title = " ".join(comment_elems[0].itertext()).strip()
                else:
                    ref_title = ""

        references[r_id] = {
            "doi":   cited_doi,
            "pmid":  cited_pmid,
            "pmcid": cited_pmcid,
            "title": ref_title
        }

    return {
        "pmid": paper_pmid,
        "doi": paper_doi,
        "pmcid": paper_pmcid,
        "title": paper_title,
        "abstract": paper_abstract,
        "references": references,
        "tree": tree
    }

##############################################################################
# 2. Minimally parse <mixed-citation> to skip authors/year/vol tags
##############################################################################

def extract_title_from_mixed_citation(mixed_elem):
    sub_article_titles = mixed_elem.xpath('.//*[local-name()="article-title"]')
    if sub_article_titles:
        return " ".join(sub_article_titles[0].itertext()).strip()

    unwanted_tags = {
        "string-name", "person-group", "year", "volume",
        "issue", "fpage", "lpage", "pub-id", "edition",
        "conf-date", "name", "isbn", "editor",
    }

    def itertext_desired(root):
        from lxml import etree
        for node in root.iter():
            if not isinstance(node, etree._Element):
                continue
            if not node.tag or not isinstance(node.tag, str):
                continue

            local = etree.QName(node).localname
            if local in unwanted_tags:
                continue

            if node.text:
                yield node.text
            if node.tail:
                yield node.tail

    parts = list(itertext_desired(mixed_elem))
    text = " ".join(p.strip() for p in parts if p.strip())
    return text

##############################################################################
# 3. get_all_citation_paragraphs: gather ALL paragraphs referencing ref_id
##############################################################################

def get_all_citation_paragraphs(tree, ref_id):
    xrefs = tree.xpath(".//xref[(@ref-type='bibr' or @ref-type='ref')]")
    matched_paras = []
    for x in xrefs:
        rid_attr = x.get("rid", "")
        rid_parts = rid_attr.split()
        if ref_id in rid_parts:
            elem = x
            while elem is not None and elem.tag != 'p':
                elem = elem.getparent()
            if elem is not None:
                paragraph_text = " ".join(elem.itertext()).strip()
                matched_paras.append(paragraph_text)
    return matched_paras

##############################################################################
# 4. Fix minor apostrophe spacing
##############################################################################

def fix_apostrophe_spacing(txt):
    """
    Insert a space if we see something like "in'burned" => "in ' burned".
    This is limited so we don't handle bigger 'typos', but it catches the
    missing space around quotes or apostrophes next to letters.
    """
    txt = re.sub(r"([A-Za-z0-9])'([A-Za-z0-9])", r"\1 ' \2", txt)
    return txt

##############################################################################
# 5. Title Normalization
##############################################################################

def normalize_title_strict(txt):
    # fix missing space around apostrophes
    txt = fix_apostrophe_spacing(txt)
    txt = txt.lower()
    # hyphens => space
    txt = re.sub(r'[-]+', ' ', txt)
    # remove punctuation except spaces
    txt = re.sub(r'[^a-z0-9\s]+', '', txt)
    # collapse multiple spaces
    txt = re.sub(r'\s+', ' ', txt)
    return txt.strip()

##############################################################################
# 6. Substring-based fallback matching approach
##############################################################################

def titles_match_substring(citing_title, cited_title):
    """
    Instead of exact equality, we check if cited_title is a substring
    of citing_title after normalizing punctuation/whitespace/hyphens.

    This helps when citing_title has extra text (authors, journal, year),
    but the actual 'paper title' is inside it.
    """
    norm_citing = normalize_title_strict(citing_title)
    norm_cited  = normalize_title_strict(cited_title)
    # Return True if norm_cited is contained in norm_citing
    return norm_cited in norm_citing

##############################################################################
# 7. Main process_pairs
##############################################################################

def process_pairs(matching_uids_xml_paths, matching_uids_pmid_list):
    """
    For each pair (xmlA, xmlB),(pmidA, pmidB):
      parse docA => dataA, docB => dataB
      check if A references B by:
        A) B's doi => revA_doi
        B) B's pmid => revA_pmid
        C) B's pmcid => revA_pmcid
        D) fallback => substring match
      if found => store paragraphs in "citation_paragraphs"
      else check B => A similarly

    We now also return indices_list, which tracks the i-th pair index
    for each found citation.
    """
    results = []
    indices_list = []

    for i, ((xmlA, xmlB), (pmidA, pmidB)) in enumerate(zip(matching_uids_xml_paths, matching_uids_pmid_list)):
        dataA = parse_nxml(xmlA)
        dataB = parse_nxml(xmlB)
        if not dataA or not dataB:
            continue

        # Quick check that dataA['pmid'] is among (pmidA, pmidB)
        if dataA['pmid'] and dataA['pmid'] not in (pmidA, pmidB):
            continue
        # Same for dataB
        if dataB['pmid'] and dataB['pmid'] not in (pmidA, pmidB):
            continue

        # Build references map for doc A
        citing_found = False
        revA_doi   = {}
        revA_pmid  = {}
        revA_pmcid = {}

        for r_id, r_info in dataA["references"].items():
            if r_info["doi"]:
                revA_doi[r_info["doi"]] = r_id
            if r_info["pmid"]:
                revA_pmid[r_info["pmid"]] = r_id
            if r_info["pmcid"]:
                revA_pmcid[r_info["pmcid"]] = r_id

        # A => B
        if dataB["doi"] and dataB["doi"] in revA_doi:
            rid = revA_doi[dataB["doi"]]
            paragraphs = get_all_citation_paragraphs(dataA["tree"], rid)
            results.append({
                "citing_title": dataA["title"],
                "citing_abstract": dataA["abstract"],
                "citation_paragraphs": paragraphs,
                "cited_title": dataB["title"],
                "cited_abstract": dataB["abstract"]
            })
            indices_list.append(i)
            citing_found = True
        elif dataB["pmid"] and dataB["pmid"] in revA_pmid:
            rid = revA_pmid[dataB["pmid"]]
            paragraphs = get_all_citation_paragraphs(dataA["tree"], rid)
            results.append({
                "citing_title": dataA["title"],
                "citing_abstract": dataA["abstract"],
                "citation_paragraphs": paragraphs,
                "cited_title": dataB["title"],
                "cited_abstract": dataB["abstract"]
            })
            indices_list.append(i)
            citing_found = True
        elif dataB["pmcid"] and dataB["pmcid"] in revA_pmcid:
            rid = revA_pmcid[dataB["pmcid"]]
            paragraphs = get_all_citation_paragraphs(dataA["tree"], rid)
            results.append({
                "citing_title": dataA["title"],
                "citing_abstract": dataA["abstract"],
                "citation_paragraphs": paragraphs,
                "cited_title": dataB["title"],
                "cited_abstract": dataB["abstract"]
            })
            indices_list.append(i)
            citing_found = True
        else:
            # fallback => substring matching for B's title
            b_title = dataB["title"].strip()
            if b_title:
                for (r_id, r_info) in dataA["references"].items():
                    # skip if r_info has the same pmid/doi/pmcid
                    if (r_info["pmid"]  == dataB["pmid"] or
                        r_info["doi"]   == dataB["doi"]  or
                        r_info["pmcid"] == dataB["pmcid"]):
                        continue
                    if titles_match_substring(r_info["title"], b_title):
                        paragraphs = get_all_citation_paragraphs(dataA["tree"], r_id)
                        results.append({
                            "citing_title": dataA["title"],
                            "citing_abstract": dataA["abstract"],
                            "citation_paragraphs": paragraphs,
                            "cited_title": dataB["title"],
                            "cited_abstract": dataB["abstract"]
                        })
                        indices_list.append(i)
                        citing_found = True
                        break

        # If not found, check B => A the same way
        if not citing_found:
            revB_doi   = {}
            revB_pmid  = {}
            revB_pmcid = {}

            for r_id, r_info in dataB["references"].items():
                if r_info["doi"]:
                    revB_doi[r_info["doi"]] = r_id
                if r_info["pmid"]:
                    revB_pmid[r_info["pmid"]] = r_id
                if r_info["pmcid"]:
                    revB_pmcid[r_info["pmcid"]] = r_id

            if dataA["doi"] and dataA["doi"] in revB_doi:
                rid = revB_doi[dataA["doi"]]
                paragraphs = get_all_citation_paragraphs(dataB["tree"], rid)
                results.append({
                    "citing_title": dataB["title"],
                    "citing_abstract": dataB["abstract"],
                    "citation_paragraphs": paragraphs,
                    "cited_title": dataA["title"],
                    "cited_abstract": dataA["abstract"]
                })
                indices_list.append(i)
            elif dataA["pmid"] and dataA["pmid"] in revB_pmid:
                rid = revB_pmid[dataA["pmid"]]
                paragraphs = get_all_citation_paragraphs(dataB["tree"], rid)
                results.append({
                    "citing_title": dataB["title"],
                    "citing_abstract": dataB["abstract"],
                    "citation_paragraphs": paragraphs,
                    "cited_title": dataA["title"],
                    "cited_abstract": dataA["abstract"]
                })
                indices_list.append(i)
            elif dataA["pmcid"] and dataA["pmcid"] in revB_pmcid:
                rid = revB_pmcid[dataA["pmcid"]]
                paragraphs = get_all_citation_paragraphs(dataB["tree"], rid)
                results.append({
                    "citing_title": dataB["title"],
                    "citing_abstract": dataB["abstract"],
                    "citation_paragraphs": paragraphs,
                    "cited_title": dataA["title"],
                    "cited_abstract": dataA["abstract"]
                })
                indices_list.append(i)
            else:
                # fallback => substring matching for A's title
                a_title = dataA["title"].strip()
                if a_title:
                    for (r_id, r_info) in dataB["references"].items():
                        if (r_info["pmid"]  == dataA["pmid"] or
                            r_info["doi"]   == dataA["doi"]  or
                            r_info["pmcid"] == dataA["pmcid"]):
                            continue
                        if titles_match_substring(r_info["title"], a_title):
                            paragraphs = get_all_citation_paragraphs(dataB["tree"], r_id)
                            results.append({
                                "citing_title": dataB["title"],
                                "citing_abstract": dataB["abstract"],
                                "citation_paragraphs": paragraphs,
                                "cited_title": dataA["title"],
                                "cited_abstract": dataA["abstract"]
                            })
                            indices_list.append(i)
                            break

    return results, indices_list


citation_list, citation_valid_indices = process_pairs(matching_uids_xml_paths, matching_uids_pmid_list)


In [27]:
len(citation_list)

31080

In [ ]:
print(f"Length of citation list: {len(citation_list)}")
print(f"Length of indices list: {len(citation_valid_indices)}")
print(f"Length of matching uids pairs: {len(matching_uids_xml_paths)}")
print(f"Difference is: {len(matching_uids_xml_paths) - len(citation_list)}")

Length of citation list: 31080
Length of indices list: 31080
Length of matching uids pairs: 31692
Difference is: 612


In [60]:
citation_list[1]

{'citing_title': 'Agenesis of permanent canines: Rare case report',
 'citing_abstract': 'Agenesis of permanent canines is a rare condition, and that of both maxillary and mandibular permanent canines is extremely rare. It may occur either isolated or in association with other dental anomalies. Reports of such cases are very scarce in the literature. Need for early diagnosis of such conditions should be emphasized because of functional, esthetic, and psychological problems which should be evaluated and treated appropriately. The present paper presents a report of bilaterally missing permanent maxillary and mandibular canines. This case might contribute in the future studies of incidence of agenesis of permanent canines.',
 'citation_paragraphs': [],
 'cited_title': 'Permanent Maxillary Canine Agenesis: A Rare Case Report',
 'cited_abstract': 'ABSTRACT Congenitally missing teeth (CMT) are among one of the commonly known dental anomalies. The most frequently missing teeth in the permanent

In [ ]:
def find_empty_citation_indices (citing_cited_articles_list):
    empty_citation_indices = []
    for idx, single_citing_cited_dict in enumerate(citing_cited_articles_list):
        if single_citing_cited_dict['citation_paragraphs'] == []:
            empty_citation_indices.append(idx)
    return empty_citation_indices

In [25]:
citation_list[0]

{'citing_title': 'Secondary choroidal neovascularization due to choroidal osteoma after 9\u2009years follow-up',
 'citing_abstract': 'Background Choroidal osteoma is a benign intraocular tumor that can increase risk of developing choroidal neovascularization. The visual prognosis is influenced by the tumor location, decalcification status, overlying RPE atrophy, presence of choroidal neovascularization, persistence of subretinal fluid and occurrence of subretinal hemorrhages. Case presentation The authors present a 40-year-old woman diagnosed with choroidal osteoma of the right eye. Her best corrected visual acuity was 12/20 but decreased to 5/20 due to secondary choroidal neovascularization after 8\u2009years follow up. Fundus examination revealed an enlarged choroidal osteoma in most margins at posterior pole with schistose hemorrhage beside macula. Optical coherence tomography angiography revealed unique features in the vascular changes of choroidal neovascularization in choroidal o

In [64]:
sublist_empty_citation_indices = find_empty_citation_indices(citation_list)

In [65]:
len(sublist_empty_citation_indices)

3407

In [23]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_extrac_list.json', 'w') as f:
#     json.dump(citation_list, f)

In [ ]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests.jsonl", "w", encoding="utf-8") as f:
    for i, record in enumerate(citation_list, start=1):
        user_content = (
            f"Citing Title: {record.get('citing_title', '')}\n"
            f"Citing Abstract: {record.get('citing_abstract', '')}\n"
            f"Citation Paragraph: {record.get('citation_paragraph', '')}\n"
            f"Cited Title: {record.get('cited_title', '')}\n"
            f"Cited Abstract: {record.get('cited_abstract', '')}"
        )

        request_obj = {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o3-mini",
                "reasoning_effort": "medium",
                "messages": [
                    {"role": "developer", "content": 
                        """You are a medical citation analysis assistant. You will read excerpts from a 'citing paper' referencing another 'cited paper,' each describing a single-patient case report. Your task is to produce four pieces of information:

                        1. Reason for Citation (a short phrase or sentence about why the authors mention the cited paper).
                        2. Possible Similarities (a brief mention of how these patients’ cases align, if at all).
                        3. Explanation (one to two sentences summarizing how the citation is used).
                        4. Confidence (High, Medium, or Low), based on the degree of similarity between the single-patient cases:
                        - High: The text explicitly states that the two patients share significant clinical or therapeutic similarities.
                        - Medium: Some parallels are hinted at but lack explicit detail or direct confirmation.
                        - Low: The citation is vague or does not clearly connect the patients’ key features.

                        ----
                        Follow this exact output format:
                        Reason for Citation: <short phrase or sentence>
                        Possible Similarities: <brief mention>
                        Explanation: <one to two sentences>
                        Confidence: <High/Medium/Low>

                        ----
                        Read the following excerpt(s) and generate your final answer in the required format. Unify your response if multiple citations are present. If no citations are specified, base your response on the articles' titles and abstracts.
                        """
                    },

                    {"role": "user", "content": user_content}
                ]
            }
        }
        # Write each object on a new line in the JSONL file
        f.write(json.dumps(request_obj, ensure_ascii=False) + "\n")

In [3]:
import json
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests.jsonl", 'r') as g:
    requests_list = [json.loads(line) for line in g]

In [4]:
len(requests_list)

31080

In [5]:
requests_list[1]

{'custom_id': 'request-2',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'o3-mini',
  'reasoning_effort': 'medium',
  'messages': [{'role': 'developer',
    'content': "You are a medical citation analysis assistant. You will read excerpts from a 'citing paper' referencing another 'cited paper,' each describing a single-patient case report. Your task is to produce four pieces of information:\n\n                        1. Reason for Citation (a short phrase or sentence about why the authors mention the cited paper).\n                        2. Possible Similarities (a brief mention of how these patients’ cases align, if at all).\n                        3. Explanation (one to two sentences summarizing how the citation is used).\n                        4. Confidence (High, Medium, or Low), based on the degree of similarity between the single-patient cases:\n                        - High: The text explicitly states that the two patients share significant clinical 

In [83]:
batch_input_file = client.files.create(
  file=open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests.jsonl", "rb"),
  purpose="batch"
)


In [148]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Citation Reason Extraction"
    }
)


Batch(id='batch_67b5f72395288190a09e7339a6dc553d', completion_window='24h', created_at=1739978531, endpoint='/v1/chat/completions', input_file_id='file-XY3rfTSNC7nLkbYGZdgfat', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1740064931, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': '0-3066'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [9]:
client.batches.retrieve("batch_67b5f72395288190a09e7339a6dc553d")

Batch(id='batch_67b5f72395288190a09e7339a6dc553d', completion_window='24h', created_at=1739978531, endpoint='/v1/chat/completions', input_file_id='file-XY3rfTSNC7nLkbYGZdgfat', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1739979745, error_file_id=None, errors=None, expired_at=None, expires_at=1740064931, failed_at=None, finalizing_at=1739979488, in_progress_at=1739978533, metadata={'description': '0-3066'}, output_file_id='file-9DD8CrVXTGvDys3Lbx76yW', request_counts=BatchRequestCounts(completed=3067, failed=0, total=3067))

In [ ]:
all_batches = client.batches.list()
all_batches

# batch_67cc7cdb7470819090d2179e8fb949fa
# file-JiUXEMpF9SW7ZprATrZRiM

SyncCursorPage[Batch](data=[Batch(id='batch_67cd5b6b42c881909c6f9730ce12b6e9', completion_window='24h', created_at=1741511531, endpoint='/v1/chat/completions', input_file_id='file-MehQVvLEtyWMwtJu83Q1eW', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1741512490, error_file_id=None, errors=None, expired_at=None, expires_at=1741597931, failed_at=None, finalizing_at=1741512473, in_progress_at=1741511532, metadata={'description': 'Batch 0'}, output_file_id='file-5d96aAgRcdTdTm1gxY7hwx', request_counts=BatchRequestCounts(completed=200, failed=0, total=200)), Batch(id='batch_67cc7cdb7470819090d2179e8fb949fa', completion_window='24h', created_at=1741454555, endpoint='/v1/chat/completions', input_file_id='file-DvvTCX1foXY75Z6spWPqKN', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1741463142, error_file_id=None, errors=None, expired_at=None, expires_at=1741540955, failed_at=None, finalizing_at=1741459623, in_pro

In [9]:
file_response = client.files.content('file-JiUXEMpF9SW7ZprATrZRiM')
file_response_text = file_response.text

json_strings = file_response_text.strip().splitlines()

# Parse each JSON string into a dictionary and store in a list
response_dicts = [json.loads(line) for line in json_strings]

o3_mini_response_list = [response_dicts[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(response_dicts))]

In [8]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/o3_mini_response_list.json', 'w') as f:
#     json.dump(o3_mini_response_list, f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/o3_mini_response_list.json', 'r') as f:
    o3_mini_response_list = json.load(f)



In [13]:
len(o3_mini_response_list)

31080

In [20]:
def split_confidence_high_med_low(response_list):
    # Initialize dictionary with default counts
    counts = {"high": 0, "medium": 0, "low": 0}
    
    for response in response_list:
        # Check if the confidence substring exists in the response
        if 'Confidence: High' in response:
            counts['high'] += 1
        elif 'Confidence: Medium' in response:
            counts['medium'] += 1
        elif 'Confidence: Low' in response:
            counts['low'] += 1
            
    return counts

split_confidence_high_med_low(o3_mini_response_list)

{'high': 6409, 'medium': 22371, 'low': 2300}

In [ ]:
o3_mini_response_list[30984][o3_mini_response_list[30984].find('Confidence'):]

'Confidence: High'

In [70]:
matching_uids_xml_paths[3]

['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC3358935/3358935_1/IJT-4-29.nxml',
 '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC3853602/3853602_1/AMHSR-3-3.nxml']

In [61]:
len(results)

7

In [32]:
matching_uids_xml_paths[0:2]

[['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC8167975/8167975_1/12886_2021_Article_2004.nxml',
  '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC5563556/5563556_1/TJO-47-243.nxml'],
 ['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4647049/4647049_1/ijcpd-08-242.nxml',
  '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC5654233/5654233_1/DRJ-14-359.nxml']]

In [9]:
import os
import glob
from lxml import etree

def extract_pmids_from_nxml(file_path):
    """Return a list of PMIDs found in a single .nxml file."""
    tree = etree.parse(file_path)
    pmid_elements = tree.xpath('//article-id[@pub-id-type="pmid"]')
    pmids = [elem.text.strip() for elem in pmid_elements if elem.text]
    return pmids[0]

def find_and_extract_pmids(directory):
    """
    Look for the first .nxml file in 'directory' (non-recursive)
    and extract PMIDs from it.
    """
    # Search only *.nxml in this directory (no subfolders)
    nxml_files = glob.glob(os.path.join(directory, '*.nxml'))
    if not nxml_files:
        return None

    # Take the first .nxml file in the directory
    file_path = nxml_files[0]

    pmc_id = file_path[61:68] + '-1'  

    # Extract PMIDs
    pmids = extract_pmids_from_nxml(file_path)
    if pmids:
        return pmids
    return None


find_and_extract_pmids('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC8167975/8167975_1')

'34058993'

In [62]:
prefix = '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2'
matching_uids_paths_list = [
    [os.path.join(prefix, os.path.join(f'PMC{id[:-2]}', f'{id[:-2]}_1')) for id in pair]
    for pair in matching_uids
]


In [63]:
matching_uids_paths_list[39]

['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC7752348/7752348_1',
 '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC7225378/7225378_1']

In [12]:
matching_uids[0:2]

[['8167975-1', '5563556-1'], ['4647049-1', '5654233-1']]

In [ ]:
# matching_uids_pmid_list = [
#     [find_and_extract_pmids(path) for path in paths_pair]
#     for paths_pair in matching_uids_paths_list
# ]

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_pmid_list.json', 'r') as f:
    matching_uids_pmid_list = json.load(f)

In [61]:
matching_uids[39]

['7752348-1', '7225378-1']

In [15]:
matching_uids_pmid_list[0:2]

[['34058993', '28845332'], ['26604546', '29109753']]

In [17]:
matching_uids_xml_paths[0]

['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC8167975/8167975_1/12886_2021_Article_2004.nxml',
 '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC5563556/5563556_1/TJO-47-243.nxml']

In [37]:
filtered_df.head(1)

,image_path,caption_path,patient_uid,pmc_id,article_path,unique_articles_sim_patients,caption_text,caption_id
0,PMC8167975/8167975_1/8167975_1_1.jpg,PMC8167975/8167975_1/8167975_1_1.txt,8167975-1,8167975,https://ncbi.nlm.nih.gov/pmc/articles/PMC8167975,['5563556-1'],"Fundus photograph showed a yellowish white, pe...",8167975_1_1


In [185]:
# # Calculating average citations per article
# sum = 0
# non_none_dict = 0

# for extraction in ref_extractions:
#     if extraction and len(extraction['paragraphs']) != 0:
#         non_none_dict += 1
#         sum += len(extraction['paragraphs'])

# avg_citations = sum / non_none_dict
# avg_citations

1.4860677840638714